# Questão 2
Utilize redes neurais perceptrons de múltiplas camadas para aproximar as funções baixo. Para o caso dos itens 2 e 3, apresente a curva da função analítica e a curva da função aproximada pela rede neural. Apresente também a curva do erro médio de treinamento com relação ao número de épocas e a curva do erro médio com o conjunto de validação. Procure definir para cada função a arquitetura da rede neural perceptron, isto é, o número de entradas, o número de neurônios em cada camada e o número de neurônios na camada de saída.
1. $f(x_{1}, x_{2}, x_{3}) = x_{1}\oplus x_{2}\oplus x_{3} \;\; x_{1}, x_{2} \text{ e } x_{3} \in \left\{0, 1\right\}$


2. $f(x) = \dfrac{\cos 2\pi x}{1-(4x)^{2}} \dfrac{\sin(\pi x)}{\pi x} \;\;,0\leq x \leq 4\pi$


3. $f(x_{1}, x_{2}) = x_{1}^{2} + x_{2}^{2} + 2x_{1}x_{2}\cos (\pi x_{1}x_{2}) + x_{1}+x_{2}-1 \;\;,|x_{1}|\leq 1 \text{ e } |x_{2}|\leq 1$

Logo abaixo estão as bibliotecas que usaremos.

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

## Solução da função 1
Vamos gerar nossos dados de treinamento e dados de validação manualmente. A função é bem simples e só admite 8 combinações de 0's e 1's.
Separaremos da seguinte forma, escolhida aleatoriamente:

<table>
  <tl>
      <td>Entradas</td>
      <td>Função</td>
  </tl>
  <tr>
    <td>000</td>
    <td>Treino</td>
  </tr>
  <tr>
    <td>001</td>
    <td>Treino</td>
  </tr>
  <tr>
    <td>010</td>
    <td>Validação</td>
  </tr>
  <tr>
    <td>011</td>
    <td>Treino</td>
  </tr>
  <tr>
    <td>100</td>
    <td>Treino</td>
  </tr>
  <tr>
    <td>101</td>
    <td>Treino</td>
  </tr>
  <tr>
    <td>110</td>
    <td>Validação</td>
  </tr>
  <tr>
    <td>111</td>
    <td>Treino</td>
  </tr>
</table>

### Construção dos vetores de treino e validação.
`x_treino` e `y_treino` serão os dados usados para treinar a rede. Já `x_teste` e `y_teste` serão usados para valida-la.

In [7]:
x = []
y = []

for i in range(2):
    for j in range(2):
        for k in range(2):
            x.append([i,j,k])
            y.append(i^j^k)

x_teste = []
x_teste.append(x[2])
x_teste.append(x[6])
y_teste = []
y_teste.append(y[2])
y_teste.append(y[6])

x_treino = [amostra for amostra in x if amostra not in x_teste]  # x_treino recebe todos os elementos de x que não estão em x_teste
y_treino = [y[i] for i in range(len(y)) if i != 2 and i != 6]  # y_treino recebe todos elementos de y menos o que estão nas posições 2 e 6

print('Tabela verdade XOR:\n[x3,x2,x1] S')
for i,j in zip(x,y):
    print(str(i)+'  '+str(j))
 

Tabela verdade XOR:
[x3,x2,x1] S
[0, 0, 0]  0
[0, 0, 1]  1
[0, 1, 0]  1
[0, 1, 1]  0
[1, 0, 0]  1
[1, 0, 1]  0
[1, 1, 0]  0
[1, 1, 1]  1


## Solução da função 2

In [ ]:
import numpy as np
import keras.layers as kl
import keras.models as km
##from tensorflow import nn  # só pra usar a ReLU e softmax
import matplotlib.pyplot as plt

def objetivo(x):
    return np.cos(2*np.pi*x)*np.sinc(x)/(1-(4*x)**2)

x_treino = np.linspace(0, 4*np.pi, 200)
y_treino = objetivo(x_treino)
plt.plot(x_treino, y_treino)
plt.show()

interv = x_treino[1] - x_treino[0]

x_teste = np.linspace(interv, 4*np.pi+interv, 200)
y_teste = objetivo(x_teste)
plt.plot(x_teste, y_teste)
plt.show()

model = km.Sequential()
model.add(kl.Dense(5, input_dim=1))
model.add(kl.Dense(5))
model.add(kl.Dense(1))

model.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])
model.fit(x_treino, y_treino, epochs=500)
#optim='sgd', loss='mse'

val_perd, val_prec = model.evaluate(x_teste, y_teste)

valores = model.predict_classes(x_teste)
